In [1]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from google.colab import drive

In [2]:
class image_data_set(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, index):
        return {'data': self.data[index], 'label': self.labels[index]}

In [10]:
def get_image_tensor(file_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    image = Image.open(file_path)
    return transform(image)

def get_data_and_labels(directory_path, label):
    image_tensors, labels = [], []
    for file in os.listdir(directory_path):
        if file.endswith(".JPG"):
            file_path = directory_path + file
            image_tensor = get_image_tensor(file_path)
            image_tensors.append(image_tensor)
            labels.append(label)
            
    return image_tensors, labels

def get_data_for_label(file_pathes, label): 

  all_data, all_labels = [], []
  for file_path in file_pathes:
      data, labels = get_data_and_labels(file_path, label)
      all_data.extend(data)
      all_labels.extend(labels)

  return all_data, all_labels
    
def get_data_sets(present_file_pathes, not_present_file_pathes): 

    present_data, present_labels = get_data_for_label(present_file_pathes, 1)
    not_present_data, not_present_labels = get_data_for_label(not_present_file_pathes, 0)
    
    print("Number of wildlife present photos: " + str(len(present_data)))
    print("Number of no wildlife present photos: " + str(len(not_present_data)))
    
    data = present_data + not_present_data
    labels = present_labels + not_present_labels
    
    training_data, testing_data, training_labels, testing_labels = train_test_split(data, labels)
    
    print("\nNumber of training photos: " + str(len(training_data)))
    print("Number of testing photos: " + str(len(testing_data)))
    
    training_data_set = image_data_set(training_data, training_labels)
    testing_data_set = image_data_set(testing_data, testing_labels)
    
    return training_data_set, testing_data_set

def get_loaders(training_data_set, testing_data_set, batch_size):
    training_loader = torch.utils.data.DataLoader(dataset = training_data_set,
                                                  batch_size = batch_size,
                                                  shuffle = True)

    testing_loader = torch.utils.data.DataLoader(dataset = testing_data_set,
                                                 batch_size = batch_size,
                                                 shuffle = True)
    
    return training_loader, testing_loader

In [4]:
def train(model, training_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    num_correct = 0
    for i, data in enumerate(training_loader):
        data, labels = data['data'].to(device), data['label'].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        running_loss += loss.item()
        _, predictions = torch.max(output.data, 1)
        num_correct += (predictions == labels).sum().item()
        loss.backward()
        optimizer.step()
    
    loss = running_loss/len(training_loader.dataset)
    accuracy = num_correct/len(training_loader.dataset)
    return loss, accuracy

def test(model, testing_loader, criterion):
    model.eval()
    running_loss = 0.0
    num_correct = 0
    for i, data in enumerate(testing_loader):
        data, labels = data['data'].to(device), data['label'].to(device)
        output = model(data)
        loss = criterion(output, labels)
        running_loss += loss.item()
        _, predictions = torch.max(output.data, 1)
        num_correct += (predictions == labels).sum().item()
    
    loss = running_loss/len(testing_loader.dataset)
    accuracy = num_correct/len(testing_loader.dataset)
    return loss, accuracy

In [5]:
def train_and_test(model, training_loader, testing_loader, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(5):
        print("epoch: " + str(epoch))
        
        training_loss, training_accuracy = train(model, training_loader, criterion, optimizer)
        print("training loss: " + str(training_loss) + " and training accuracy: " + str(training_accuracy))
        
        testing_loss, testing_accuracy = test(model, testing_loader, criterion)
        print("testing loss: " + str(testing_loss) + " and testing accuracy: " + str(testing_accuracy))

In [6]:
def train_and_test_VGG11(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing VGG11")
    vgg11 = models.vgg11(weights = models.VGG11_Weights.DEFAULT)
    vgg11.classifier[6].out_features = num_classes
    train_and_test(vgg11, training_loader, testing_loader, device)

def train_and_test_VGG19(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing VGG19")
    vgg19 = models.vgg19(weights = models.VGG19_Weights.DEFAULT)
    vgg19.classifier[6].out_features = num_classes
    train_and_test(vgg19, training_loader, testing_loader, device)

def train_and_test_ResNet50(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing ResNet50")
    resnet50 = models.resnet50(weights = models.ResNet50_Weights.DEFAULT)
    resnet50.fc.out_features = num_classes
    train_and_test(resnet50, training_loader, testing_loader, device)

def train_and_test_ResNet152(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing ResNet152")
    resnet152 = models.resnet152(weights = models.ResNet152_Weights.DEFAULT)
    resnet152.fc.out_features = num_classes
    train_and_test(resnet152, training_loader, testing_loader, device)
    
def train_and_test_AlexNet(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing AlexNet")
    alexnet = models.alexnet(weights = models.AlexNet_Weights.DEFAULT)
    alexnet.classifier[6].out_features = num_classes
    train_and_test(alexnet, training_loader, testing_loader, device)

def train_and_test_SSD(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing SSD")
    ssd300_vgg16 = models.detection.ssd300_vgg16(weights = models.detection.SSD300_VGG16_Weights.COCO_V1)
    train_and_test(ssd300_vgg16, training_loader, testing_loader, device)
    
def train_and_test_Faster_RCNN(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing Faster R-CNN")
    faster_rcnn = models.detection.fasterrcnn_resnet50_fpn_v2(weights = models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
    faster_rcnn.roi_heads.box_predictor.bbox_pred.out_features = num_classes
    train_and_test(faster_rcnn, training_loader, testing_loader, device)
    
def train_and_test_YOLO(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing YOLO")
    yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    train_and_test(yolo, training_loader, testing_loader, device)

#Orchestration

In [7]:
# Use this to connect to Google Drive in Google Colab
drive.mount('/content/drive')

# Use this to unzip file in Google Colab
!unzip drive/MyDrive/manually_labeled_wildlife_data

Mounted at /content/drive
Archive:  drive/MyDrive/manually_labeled_wildlife_data.zip
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/animal/05_09_13 (11).JPG  
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/animal/05_09_13 (15).JPG  
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/animal/05_09_13 (12).JPG  
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/animal/05_09_13 (13).JPG  
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/animal/05_09_13 (1).JPG  
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/animal/05_09_13 (10).JPG  
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/animal/05_09_13 (14).JPG  
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/animal/05_09_13 (17).JPG  
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/animal/05_09_13 (18).JPG  
  inflating: manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/ani

In [11]:
present_file_pathes = [
  "manually_labeled_wildlife_data/MP152_ODOT009_REPELCAM/present/",
  "manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/present/"
]

not_present_file_pathes = [
  "manually_labeled_wildlife_data/MP152_ODOT009_REPELCAM/not_present/",
  "manually_labeled_wildlife_data/MP152_ODOT003_EASTFACE/not_present/",
]

num_classes = 2
batch_size = 10

training_data_set, testing_data_set = get_data_sets(present_file_pathes, not_present_file_pathes)
training_loader, testing_loader = get_loaders(training_data_set, testing_data_set, batch_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

Number of wildlife present photos: 945
Number of no wildlife present photos: 3050

Number of training photos: 2996
Number of testing photos: 999


In [12]:
train_and_test_VGG11(training_loader, testing_loader, device, num_classes)


Training and Testing VGG11


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

epoch: 0
training loss: 0.038904552596486924 and training accuracy: 0.8858477970627503
testing loss: 0.017015286532041426 and testing accuracy: 0.9409409409409409
epoch: 1
training loss: 0.015016122634620691 and training accuracy: 0.9385847797062751
testing loss: 0.016455470448675298 and testing accuracy: 0.9329329329329329
epoch: 2
training loss: 0.012029995879164123 and training accuracy: 0.9499332443257676
testing loss: 0.01097445633213802 and testing accuracy: 0.953953953953954
epoch: 3
training loss: 0.009794916347048363 and training accuracy: 0.9596128170894526
testing loss: 0.012310160678427311 and testing accuracy: 0.9569569569569569
epoch: 4
training loss: 0.009244082693056817 and training accuracy: 0.9636181575433912
testing loss: 0.014732727435246404 and testing accuracy: 0.9569569569569569


In [13]:
train_and_test_VGG19(training_loader, testing_loader, device, num_classes)


Training and Testing VGG19


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

epoch: 0
training loss: 0.03588738656290468 and training accuracy: 0.883177570093458
testing loss: 0.013175265891621966 and testing accuracy: 0.94994994994995
epoch: 1
training loss: 0.016245085222114146 and training accuracy: 0.931909212283044
testing loss: 0.013458543568145711 and testing accuracy: 0.948948948948949
epoch: 2
training loss: 0.010761466990069201 and training accuracy: 0.9532710280373832
testing loss: 0.011334580996875988 and testing accuracy: 0.9459459459459459
epoch: 3
training loss: 0.007905529706449482 and training accuracy: 0.9669559412550067
testing loss: 0.009951238354137918 and testing accuracy: 0.9579579579579579
epoch: 4
training loss: 0.00799754288364202 and training accuracy: 0.9646194926568759
testing loss: 0.008149764191035207 and testing accuracy: 0.95995995995996


In [14]:
train_and_test_ResNet50(training_loader, testing_loader, device, num_classes)


Training and Testing ResNet50


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

epoch: 0
training loss: 0.06485395707396445 and training accuracy: 0.8734979973297731
testing loss: 0.012876583317348788 and testing accuracy: 0.94994994994995
epoch: 1
training loss: 0.010259941920718284 and training accuracy: 0.9576101468624834
testing loss: 0.010985146744644738 and testing accuracy: 0.948948948948949
epoch: 2
training loss: 0.005894174085521526 and training accuracy: 0.9779706275033377
testing loss: 0.009989762568037148 and testing accuracy: 0.9579579579579579
epoch: 3
training loss: 0.005876105763431569 and training accuracy: 0.9766355140186916
testing loss: 0.008768432584518494 and testing accuracy: 0.968968968968969
epoch: 4
training loss: 0.00416515490621638 and training accuracy: 0.9823097463284379
testing loss: 0.006520723738696881 and testing accuracy: 0.973973973973974


In [15]:
train_and_test_ResNet152(training_loader, testing_loader, device, num_classes)


Training and Testing ResNet152


Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

epoch: 0
training loss: 0.057873387603419965 and training accuracy: 0.8781708945260347
testing loss: 0.012080525243384493 and testing accuracy: 0.9569569569569569
epoch: 1
training loss: 0.009179890566742985 and training accuracy: 0.9659546061415221
testing loss: 0.009147607187206322 and testing accuracy: 0.96996996996997
epoch: 2
training loss: 0.006106207772992422 and training accuracy: 0.9753004005340454
testing loss: 0.008476396625813981 and testing accuracy: 0.963963963963964
epoch: 3
training loss: 0.004297895551103046 and training accuracy: 0.9819759679572764
testing loss: 0.007773572955394977 and testing accuracy: 0.973973973973974
epoch: 4
training loss: 0.0031370486909540625 and training accuracy: 0.9873164218958611
testing loss: 0.00727200697825707 and testing accuracy: 0.978978978978979


In [16]:
train_and_test_AlexNet(training_loader, testing_loader, device, num_classes)


Training and Testing AlexNet


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

epoch: 0
training loss: 0.03636053994548064 and training accuracy: 0.897196261682243
testing loss: 0.015036431581455024 and testing accuracy: 0.9419419419419419
epoch: 1
training loss: 0.017737575398621433 and training accuracy: 0.9305740987983978
testing loss: 0.012954643415240524 and testing accuracy: 0.94994994994995
epoch: 2
training loss: 0.01285029785462095 and training accuracy: 0.9499332443257676
testing loss: 0.015231995445110561 and testing accuracy: 0.9409409409409409
epoch: 3
training loss: 0.012392420470583233 and training accuracy: 0.9522696929238985
testing loss: 0.01228948371738682 and testing accuracy: 0.9529529529529529
epoch: 4
training loss: 0.011130636014381835 and training accuracy: 0.9569425901201603
testing loss: 0.011631973151559685 and testing accuracy: 0.9519519519519519


In [17]:
#TODO: Get these working
#train_and_test_Faster_RCNN(training_loader, testing_loader, device, num_classes)
#train_and_test_YOLO(training_loader, testing_loader, device, num_classes)
#train_and_test_SSD(training_loader, testing_loader, device, num_classes)